# Task 1

Build the code to scrape postcodes from the wikipedia page

The postal codes table is scraped from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup

In [2]:
!wget -q -O wikiToronto.html https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
with open("wikiToronto.html", encoding="utf-8") as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
text_table = ''
count = 1
for Postcode in soup.tbody.find_all('td'):
    if count == 3:
        count = 1
        text_table = text_table + Postcode.text
    else:
        count+= 1
        text_table = text_table + Postcode.text + ','
        
print(text_table)

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Downtown Toronto,Queen's Park
M8A,Not assigned,Not assigned
M9A,Queen's Park,Not assigned
M1B,Scarborough,Rouge
M1B,Scarborough,Malvern
M2B,Not assigned,Not assigned
M3B,North York,Don Mills North
M4B,East York,Woodbine Gardens
M4B,East York,Parkview Hill
M5B,Downtown Toronto,Ryerson
M5B,Downtown Toronto,Garden District
M6B,North York,Glencairn
M7B,Not assigned,Not assigned
M8B,Not assigned,Not assigned
M9B,Etobicoke,Cloverdale
M9B,Etobicoke,Islington
M9B,Etobicoke,Martin Grove
M9B,Etobicoke,Princess Gardens
M9B,Etobicoke,West Deane Park
M1C,Scarborough,Highland Creek
M1C,Scarborough,Rouge Hill
M1C,Scarborough,Port Union
M2C,Not assigned,Not assigned
M3C,North York,Flemingdon Park
M3C,North York,Don Mills South
M4C,East York,Woodbine Heights
M5C,Downtown Toronto,St. James 

In [4]:
file=open("toronto_postcode.csv","wb")
file.write(bytes(text_table,encoding="ascii",errors="ignore"))
file.close()

In [5]:
column_names = soup.find('tr')
column_names

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

In [6]:
column_names = ["Postcode", "Borough", "Neighbourhood"]

In [7]:
# convert csv to dataframe
df = pd.read_csv('toronto_postcode.csv',header=None) 
df.columns= column_names
df.head(9)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned


In [8]:
df_g = df.groupby(['Postcode', 'Borough'], sort = False).agg( ', '.join)
df_g_i = df_g.reset_index()
df_g_i.head(9)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park
7,M8A,Not assigned,Not assigned
8,M9A,Queen's Park,Not assigned


In [9]:
# delete Not assigned rows
not_assigned = df_g_i[df_g_i["Borough"] == "Not assigned"].index
df_g_i.drop(not_assigned, inplace = True)
df_g_i.head(9)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park
8,M9A,Queen's Park,Not assigned
9,M1B,Scarborough,"Rouge, Malvern"
11,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens, Parkview Hill"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

**Note: In row 8, Borough is assigned as Queen's Park, but Neighbouthood is assigned as Not assigned.**

DataFrame.loc access a group of rows and columns by label(s) or a boolean array.

In [10]:
df_g_i.loc[df_g_i["Neighbourhood"] == "Not assigned", "Neighbourhood"] = df_g_i["Borough"]
df_g_i.head(9)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,"Lawrence Heights, Lawrence Manor"
6,M7A,Downtown Toronto,Queen's Park
8,M9A,Queen's Park,Queen's Park
9,M1B,Scarborough,"Rouge, Malvern"
11,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens, Parkview Hill"


In [11]:
# reset the index and drop the orginal index
df_f = df_g_i.reset_index(drop=True)
df_f.head(9)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park
5,M9A,Queen's Park,Queen's Park
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"


Use the .shape method to print the number of rows of your dataframe.

In [12]:
df_f.shape

(103, 3)

# Task 2

Use the Geocoder package or the csv file to create the following dataframe

In [13]:
!wget -q -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data
print("CSV file downloaded")

CSV file downloaded


In [14]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv")
df_geo.head(9)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476


In [15]:
# rename the Postal Code (set inplace = True to make to the existing DataFrame)
df_geo.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
df_geo.head(9)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476


Merge two tables

on: Column or index level names to join on.

In [16]:
df_merged = df_f.merge(df_geo, on = "Postcode")
df_merged.head(9)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
5,M9A,Queen's Park,Queen's Park,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937


# Task 3

Explore and cluster the neighborhoods in Toronto.
1. To add enough Markdown cells to explain what you decided to do and to report any observations you make.
2. To generate maps to visualize your neighborhoods and how they cluster together

In [17]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium
import requests # library to handle requests

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

# All requested packages already installed.



Explore neighbourhoods in Downtown Toronto

In [18]:
df_downtown = df_merged[df_merged["Borough"] =="Downtown Toronto"].reset_index(drop = True)
df_downtown

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
9,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576


In [19]:
df_downtown_1  = df_downtown[["Neighbourhood", "Latitude", "Longitude"]]
df_downtown_1.head()

,Neighbourhood,Latitude,Longitude
0,Harbourfront,43.654260,-79.360636
1,Queen's Park,43.662301,-79.389494
2,"Ryerson, Garden District",43.657162,-79.378937
3,St. James Town,43.651494,-79.375418
4,Berczy Park,43.644771,-79.373306


In [20]:
Latitude = 43.706397
Longitude = -79.309937
map_toronto= folium.Map(location=[Latitude, Longitude], zoom_start=13)

for lat, lng, label in zip(df_downtown_1["Latitude"], df_downtown_1["Longitude"], df_downtown_1["Neighbourhood"]):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 

map_toronto


#### Define Foursquare Credentials and Version

In [21]:
# The code was removed by Watson Studio for sharing.

Get the top 100 venues that are in Berczy Park with 2000m radius

In [22]:
neighborhood_latitude = df_downtown_1.loc[4, "Latitude"]
neighborhood_longitude  = df_downtown_1.loc[4, "Longitude"]

radius = 2000 # define radius
LIMIT = 100 # limit the number of venues returned by Foursquare API
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
print(url)

https://api.foursquare.com/v2/venues/explore?&client_id=PXG44EWA2LZLHSLW4HGCFMNKPMYXQSGLKHYSZMOIJLOUGDVM&client_secret=RMHL1N224V35L5KV101HOVHS3P5EC5UAYJBO54Y3Y145ZO0Q&v=20180605&ll=43.644770799999996,-79.3733064&radius=2000&limit=100


In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e481c76542890001b15be5f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Downtown Toronto',
  'headerFullLocation': 'Downtown Toronto, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 242,
  'suggestedBounds': {'ne': {'lat': 43.66277081800001,
    'lng': -79.34847832342174},
   'sw': {'lat': 43.62677078199998, 'lng': -79.39813447657826}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b56a44ff964a5206e1728e3',
       'name': 'The Keg Steakhouse + Bar',
       'location': {'address': '26 The Esplanade',
        'lat': 43.64667637593993,
        'lng': -79.37482154865866,
        'labeledLatLngs': [{'label': 'display

In [24]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,The Keg Steakhouse + Bar,Steakhouse,43.646676,-79.374822
1,Fresh On Front,Vegetarian / Vegan Restaurant,43.647815,-79.374453
2,St. Lawrence Market (South Building),Farmers Market,43.648743,-79.371597
3,Hockey Hall Of Fame (Hockey Hall of Fame),Museum,43.646974,-79.377323
4,LCBO,Liquor Store,43.642944,-79.372440
5,St. Lawrence Market (North Building),Farmers Market,43.648793,-79.371945
6,Hogtown Smoke,Food Truck,43.649287,-79.374689
7,Miku,Japanese Restaurant,43.641374,-79.377531
8,DAVIDsTEA,Tea Room,43.646658,-79.377641
9,Berczy Park,Park,43.648048,-79.375172


In [26]:
categories = nearby_venues.groupby(["categories"]).count()
categories_count = categories.rename(columns = {"name": "count"}).reset_index()
c_count = categories_count[["categories", "count"]]
c_count

,categories,count
0,American Restaurant,2
1,Aquarium,1
2,BBQ Joint,1
3,Bakery,2
4,Basketball Stadium,1
5,Beer Bar,2
6,Bistro,2
7,Breakfast Spot,1
8,Brewery,1
9,Café,5


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
downtown_venues = getNearbyVenues(names=df_downtown_1['Neighbourhood'],
                                   latitudes=df_downtown_1['Latitude'],
                                   longitudes=df_downtown_1['Longitude']
                                  )

Harbourfront
Queen's Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
First Canadian Place, Underground city
Church and Wellesley


In [29]:
print(downtown_venues.shape)
downtown_venues.head()

(1676, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [30]:
downtown_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",37,37,37,37,37,37
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [31]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']], prefix="", prefix_sep="")

downtown_onehot["Neighbourhood"] = downtown_venues ["Neighbourhood"]

# move neighbourhood column (which is the last one index -1) to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]
downtown_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [32]:
downtown_onehot_grouped = downtown_onehot.groupby('Neighbourhood').mean().reset_index()
downtown_onehot_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,South American Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.0000,0.0000,0.02,0.00,0.00,0.00,0.02,0.01,0.000000,0.00,0.000000,0.00,0.030000,0.000000,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.000000,0.01,0.01,0.020000,0.00,0.00,0.01,0.01,0.01,0.000000,0.060000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.02,0.00,0.050000,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.00,0.00,0.000000,0.010000,0.01,0.00,0.00,0.000000,0.0000,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.01,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.020000,0.00,0.00,0.00,0.00,0.000000,0.01,0.00,0.01,0.000000,0.02,0.01,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.000000,0.00,0.05,0.00,0.00,0.00,0.000000,0.00,0.010000,0.020000,0.01,0.000000,0.00,0.010000,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.000000,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,0.01,0.01,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.02,0.000000,0.01,0.00,0.000000,0.00,0.020000,0.01,0.00,0.030000,0.00,0.000000,0.00,0.00,0.01,0.01,0.000000,0.0000,0.00,0.0000,0.02,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.00,0.020000,0.00,0.00,0.03,0.01,0.

In [33]:
downtown_onehot_grouped.shape

(19, 205)

The function to sort the venues in descending order

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
downtown_onehot_grouped.iloc[0]

Neighbourhood                    Adelaide, King, Richmond
Afghan Restaurant                                       0
Airport                                                 0
Airport Lounge                                          0
American Restaurant                                  0.02
Animal Shelter                                          0
Aquarium                                                0
Art Gallery                                             0
Arts & Crafts Store                                  0.02
Asian Restaurant                                     0.01
Athletics & Sports                                      0
Auto Dealership                                         0
BBQ Joint                                               0
Bagel Shop                                              0
Bakery                                               0.03
Bank                                                    0
Bar                                                  0.01
Baseball Stadi

In [36]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = downtown_onehot_grouped['Neighbourhood']

for ind in np.arange(downtown_onehot_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_onehot_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Café,Hotel,Coffee Shop,Theater,Restaurant
1,Berczy Park,Coffee Shop,Café,Hotel,Park,Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Harbor / Marina,Airport,Airport Lounge
3,"Cabbagetown, St. James Town",Café,Gastropub,Park,Restaurant,Coffee Shop
4,Central Bay Street,Coffee Shop,Japanese Restaurant,Ramen Restaurant,Ice Cream Shop,Café


In [37]:
kclusters = 6

downtown_grouped_clustering = downtown_onehot_grouped.drop('Neighbourhood', 1)


kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

kmeans.labels_[0:10] 


array([2, 2, 3, 4, 0, 5, 5, 0, 2, 2], dtype=int32)

In [ ]:
# add clustering labels
df_downtown_1.insert(0, 'Cluster Labels', kmeans.labels_)

In [41]:
downtown_merged = df_downtown_1
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

downtown_merged.head() # check the last columns!

,Cluster Labels,Neighbourhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,2,Harbourfront,43.654260,-79.360636,Coffee Shop,Café,Park,Pub,Diner
1,2,Queen's Park,43.662301,-79.389494,Coffee Shop,Park,Gastropub,Japanese Restaurant,Sushi Restaurant
2,3,"Ryerson, Garden District",43.657162,-79.378937,Coffee Shop,Clothing Store,Diner,Restaurant,Café
3,4,St. James Town,43.651494,-79.375418,Café,Coffee Shop,Bakery,Restaurant,Breakfast Spot
4,0,Berczy Park,43.644771,-79.373306,Coffee Shop,Café,Hotel,Park,Restaurant


In [42]:
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighbourhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters